In [ ]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from bs4 import NavigableString
from bs4 import Comment
import sys
from urllib.parse import urlparse
import random
import time

In [ ]:
#all_users = []

In [ ]:
medium_users = set()

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
}

sitemap = 'https://medium.com/sitemap/posts/2018/posts-2018-09-25.xml'

response = requests.get(sitemap, headers=headers)
root = ET.fromstring(response.content)

In [ ]:
for child in root:
    for grandchild in child:
        if grandchild.tag == '{http://www.sitemaps.org/schemas/sitemap/0.9}loc':
            url_parts = urlparse(grandchild.text)
            path_parts = url_parts.path.split('/')
            if len(path_parts) >= 2:
                if path_parts[1][0] == '@':
                    medium_users.add(path_parts[1])

In [ ]:
len(medium_users)

In [ ]:
def add_user_info(username):
    user = {}
    user['username'] = username

    response = requests.get('https://medium.com/' + username, headers)
    #response = requests.get('https://medium.com/@Wilkner', headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    user['name'] = soup.h1.contents

    possible_description = soup.h1.parent.next_sibling.find('p')
    if possible_description is not None:
        user['description'] = possible_description.contents

    links_container = soup.h1.parent.next_sibling.next_sibling.next_sibling
    if links_container is not None:
        links = links_container.find_all('a')

        if links is not None:
            for link_num, link in enumerate(links):
                number = False
                if link_num == 0 or link_num == 1:
                    for i, aspect in enumerate(link):
                        if i == 0:
                            number = str(aspect)
                        if i == 2:
                            if 'Following' in aspect:
                                user['following'] = number
                            if 'Followers' in aspect:
                                user['followers'] = number
                if link_num == 2:
                    user['external_link'] = link['href']
    all_users.append(user)
    sleep(5)
    return

In [ ]:
#for i, username in enumerate(random.sample(medium_users, 10)):
for i, username in enumerate(medium_users):
    try:
        found = False
        
        for existing_user in all_users:
            if existing_user['username'] == username:
                found = True
        
        if found == False:
            print(len(all_users))
            add_user_info(username)
    except:
        pass

In [25]:
all_users

[{'username': '@aleksandrstepanovx',
  'name': ['Aleksandr Stepanov'],
  'description': [],
  'following': '5'},
 {'username': '@romaneuuushilov',
  'name': ['Знойный чпок'],
  'description': [],
  'following': '1'},
 {'username': '@lizabkt',
  'name': ['Lisa B.'],
  'description': [],
  'following': '3',
  'followers': '2'},
 {'username': '@craig_10243',
  'name': ['Craig Wright'],
  'description': ['My opinions are my own. Eternal student & researcher; plugging Bitcoin from as long as it was lawyer, banker, economist, coder, investor, mathematician, & stats'],
  'following': '2',
  'followers': '513'},
 {'username': '@netkoin',
  'name': ['Netkoin'],
  'description': ['Reinventing the retail business and customer relationship.'],
  'followers': '1.2K'},
 {'username': '@aurora_8light',
  'name': ['Maria Lotsman'],
  'description': [],
  'following': '95',
  'followers': '62'},
 {'username': '@5thFear',
  'name': ['The Crypto Guy'],
  'description': [],
  'following': '24',
  'follower

In [67]:
import csv

keys = all_users[0].keys()
with open('mediumprofilesx.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_users)


ValueError: dict contains fields not in fieldnames: 'followers'

In [61]:
len(all_users)

11412